In [1]:
from biophi.humanization.methods.sapiens.predict import sapiens_predict_seq, sapiens_predict_chain
import pandas as pd
import numpy as np
import torch
from abnumber import Chain

In [2]:
def get_embeds(chain):
    pred, extra = sapiens_predict_chain(
        chain=chain,
        return_all_hiddens=True
    )
    return extra

In [3]:
def embed_seq(seq):
    if len(seq) > 142:
        seq = seq[:142]
    if seq == "QVQLQESGPRLVKPSDTLSLTCTVSGGSITSDSHYWGWVRQSPGKGLEWIASTSDSGSTYHNRPIYYNPSLKSRVTISVATSKNQFSLKLRSVTAADTAMYYCVRQWRYSSGSLDCWGQGILVTVSS" or seq == "EPQLQESGPTLVEASETLSLTCAVSGDSTAACNSFWGWVRQPPGKGLEWVGSLSHCASYWNRGWTYHNPSLKSRLTLALDTPKNLVFLKLNSVTAADTATYYCARFGGEVLRYTDWPKPAWVDLWGRGTLVTVSS":
        pred, extra = sapiens_predict_seq(seq, chain_type="H", model_version='latest', return_all_hiddens=True)
    else:
        ch = Chain(seq, "imgt")
        extra = get_embeds(ch)
    fourth = extra['inner_states'][3]
    fourth_reduced = torch.mean(fourth, (0, 1))
    
    fifth = extra['inner_states'][4]
    fifth_reduced = torch.mean(fifth, (0, 1))
    
    return fourth_reduced.numpy(), fifth_reduced.numpy()

In [4]:
chen_data = pd.read_csv("../diplomka/di-pred/data/chen/deduplicated/chen_data.csv", index_col=0)
chen_data.head()

,Antibody_ID,heavy,light,Y
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0


In [5]:
tap_data = pd.read_csv("../diplomka/di-pred/data/tap/TAP_data.csv")
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [6]:
chen_data["h_embed"] = chen_data["heavy"].apply(embed_seq)
chen_data["l_embed"] = chen_data["light"].apply(embed_seq)
chen_data.head()

,Antibody_ID,heavy,light,Y,h_embed,l_embed
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0,"([-0.6926601, 0.042352896, 0.51911086, 0.22290...","([0.47513103, 0.55213594, -0.40027273, -0.1948..."
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0,"([-0.7220113, 0.07192377, 0.46017027, 0.116428...","([0.0027824142, -0.31247252, -0.2616108, -0.05..."
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1,"([-0.68661505, -0.08617489, 0.52407867, 0.1730...","([0.5381147, 0.5266551, -0.7495708, -0.2031394..."
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0,"([-0.6857544, -0.017793871, 0.6010467, 0.05745...","([0.51654476, 0.6286876, -0.75871503, -0.23285..."
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0,"([-0.00818961, -0.14749222, 0.36846554, -0.264...","([0.63308156, 0.38202587, -0.52824193, -0.2208..."


In [ ]:
chen_data.to_csv("../diplomka/di-pred/data/chen/embeddings/sapiens/chen_data_embedded.csv")

In [4]:
import pandas as pd
import numpy as np
from os import path

In [6]:
DATA_DIR = "../../data"

In [8]:
chen_data = pd.read_csv(path.join(DATA_DIR, "chen/embeddings/sapiens/chen_data_embedded.csv"), index_col=0)
chen_data.head()

,Antibody_ID,heavy,light,Y,h_embed,l_embed
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0,"(array([-0.6926601 , 0.0423529 , 0.51911086,...","(array([ 0.47513103, 0.55213594, -0.40027273,..."
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0,"(array([-0.7220113 , 0.07192377, 0.46017027,...","(array([ 0.00278241, -0.31247252, -0.2616108 ,..."
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1,"(array([-6.86615050e-01, -8.61748904e-02, 5.2...","(array([ 5.3811473e-01, 5.2665508e-01, -7.495..."
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0,"(array([-0.6857544 , -0.01779387, 0.6010467 ,...","(array([ 0.51654476, 0.6286876 , -0.75871503,..."
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0,"(array([-0.00818961, -0.14749222, 0.36846554,...","(array([ 6.33081555e-01, 3.82025868e-01, -5.2..."


In [12]:
embeds_str = chen_data.iloc[0]["h_embed"]

In [28]:
def parse_embed_arrays(string, which):
    parts = string.split("array")
    return parts[which + 1]

In [20]:
def remove_weird_chars(chars, string):
    for char in chars:
        string = string.replace(char, "")
    return string

In [33]:
def embeds_from_string(row, which=1, chain="h"):
    string = row[f"{chain}_embed"]
    string = parse_embed_arrays(string, which)
    weird_chars = ["(", ")", "[", "]", ",", "\n", "'", "dtype=float32"]
    string = remove_weird_chars(weird_chars, string)
    parts = string.split()
    return [float(num) for num in parts]

In [31]:
len(embeds_from_string(chen_data.iloc[0]["h_embed"]))

128

In [35]:
# first with last layer (second array)
embeddings_h = chen_data.apply(embeds_from_string, which=1, chain="h", result_type="expand", axis=1)

In [36]:
embeddings_h

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.132917,0.076576,0.380830,0.266019,0.136909,-0.066856,-0.014169,-0.396702,-0.023894,0.049665,...,-0.157948,-0.221006,0.223824,0.111430,0.322759,-0.420418,-0.505346,-0.032005,-3.020458,-0.093679
1,-0.184553,0.158254,0.355150,0.200379,0.058060,-0.038602,0.090086,-0.406580,-0.015033,0.001473,...,-0.181082,-0.184320,0.259686,0.148743,0.311722,-0.350911,-0.561503,0.016358,-3.014494,-0.196574
2,-0.212230,0.059972,0.404980,0.231210,0.109703,-0.010467,0.018848,-0.444627,-0.062082,0.052569,...,-0.215605,-0.225568,0.206208,0.138171,0.290136,-0.388805,-0.564260,0.020575,-3.008310,-0.159851
3,-0.154947,0.099448,0.392644,0.179663,0.143432,-0.016784,0.074854,-0.401143,-0.064605,0.030262,...,-0.214371,-0.226481,0.203215,0.136327,0.286761,-0.289681,-0.524033,0.037393,-3.011721,-0.126978
4,0.240583,-0.096983,0.339282,-0.142076,0.024596,-0.216692,0.119942,-0.422370,0.094399,-0.002828,...,-0.176714,-0.041248,0.359067,-0.098987,-0.072250,-0.277831,-0.507991,-0.161195,-3.022917,0.023900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2404,-0.022100,-0.011111,0.305766,0.224684,-0.016072,0.082764,-0.008633,-0.312531,-0.320898,-0.069881,...,-0.512476,-0.304521,0.279696,-0.090272,0.036613,-0.237879,-0.518702,0.032643,-3.019043,-0.214922
2405,-0.022128,0.128562,0.524840,0.327025,0.277255,-0.139878,-0.002099,-0.506137,-0.017233,-0.078609,...,-0.398623,-0.424670,0.341347,0.190503,0.212816,-0.523696,-0.716431,-0.221871,-3.006454,0.028419
2406,0.106308,-0.080380,0.419436,0.109451,0.179123,0.264232,-0.037324,-0.490189,-0.357727,-0.010142,...,-0.557391,-0.396221,0.312066,-0.027133,-0.127454,-0.177076,-0.598729,-0.041176,-3.033348,-0.162813
2407,0.210198,-0.116412,0.317396,-0.114860,-0.030604,-0.251481,0.136465,-0.434119,0.033934,-0.035255,...,-0.057816,-0.013492,0.386436,-0.077789,-0.118081,-0.054696,-0.513055,-0.198051,-3.024876,0.048959


In [37]:
embeddings_l = chen_data.apply(embeds_from_string, which=1, chain="l", result_type="expand", axis=1)

In [44]:
embeddings = pd.concat([embeddings_h, embeddings_l], axis=1)
embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-0.132917,0.076576,0.380830,0.266019,0.136909,-0.066856,-0.014169,-0.396702,-0.023894,0.049665,...,-0.556073,-0.650359,0.332396,0.633688,0.246616,-1.158941,-0.102747,-0.341558,-0.683722,-0.289992
1,-0.184553,0.158254,0.355150,0.200379,0.058060,-0.038602,0.090086,-0.406580,-0.015033,0.001473,...,0.057405,0.739740,0.179962,0.235049,1.009239,-1.214688,-0.357766,-1.223195,-0.823662,0.093647
2,-0.212230,0.059972,0.404980,0.231210,0.109703,-0.010467,0.018848,-0.444627,-0.062082,0.052569,...,-0.301882,-0.580996,-0.002240,0.726648,0.300416,-1.255112,-0.100235,-0.203760,-0.434915,-0.134284
3,-0.154947,0.099448,0.392644,0.179663,0.143432,-0.016784,0.074854,-0.401143,-0.064605,0.030262,...,-0.450211,-0.661695,0.086359,0.952898,0.289991,-1.225952,-0.319906,-0.390223,-0.598955,-0.218546
4,0.240583,-0.096983,0.339282,-0.142076,0.024596,-0.216692,0.119942,-0.422370,0.094399,-0.002828,...,-0.510612,-0.366230,0.203418,0.751161,0.595421,-1.178856,-0.296510,-0.212104,-0.641828,-0.139706


In [45]:
embeddings.insert(0, "Ab_ID", chen_data["Antibody_ID"])

In [47]:
embeddings.to_csv(path.join(DATA_DIR, "chen/embeddings/sapiens/chen_data_embedded_clean.csv"))